<a href="https://colab.research.google.com/github/ahairshi/dsf-redis/blob/main/I_need_a_single_file%2C_and_need_to_run_for_request_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Achieving a ramp-up to **900,000 requests per minute (RPM)** requires a massive amount of throughput, which translates to a high **hatch rate** (users spawned per second) and likely a large number of total concurrent users.

Here is the updated Locust script, configured to handle the XML payloads from files, use the high-performance `FastHttpUser`, and includes the recommended command-line options for such a high-scale, fast-ramping test.

-----

## Locust Script for High-Volume XML Load Test (`locustfile.py`)

This script is identical to the previous version but includes notes on how to run it for your specific ramp-up requirement.

In [ ]:
import os
import random
from locust import FastHttpUser, task, between, events
from typing import Dict, List, Any

# ==============================================================================
# 1. Configuration & Data Loading
# ==============================================================================

# Define the list of XML payload files to load
XML_FILES: List[str] = [
    "payload_a.xml",
    "payload_b.xml"
]

# This will store the content of the XML files
PAYLOADS: List[str] = []

# Define the headers (replace with your actual cURL headers)
HEADERS: Dict[str, str] = {
    'Content-Type': 'text/xml; charset=utf-8',
    'SOAPAction': '"http://tempuri.org/MethodName"',
    'Accept-Encoding': 'gzip,deflate'
}

# --- Event Hook to Load Files Before Test Starts ---
@events.init.add_listener
def _(environment, **kwargs):
    """Load all XML payload files into memory when Locust starts."""
    for file_name in XML_FILES:
        try:
            # Load file content once
            file_path = os.path.join(os.path.dirname(__file__), file_name)
            with open(file_path, 'r', encoding='utf-8') as f:
                PAYLOADS.append(f.read())
            print(f"INFO: Successfully loaded payload from {file_name}")

        except FileNotFoundError:
            print(f"ERROR: Payload file not found: {file_name}")
            # Consider raising an exception here to halt the test if files are critical
        except Exception as e:
            print(f"ERROR: Could not read file {file_name}: {e}")

# ==============================================================================
# 2. Locust User Class Definition using FastHttpUser
# ==============================================================================

class XmlServiceUser(FastHttpUser):
    """Simulates a user making requests with a randomly selected XML payload."""

    # !!! IMPORTANT: SET YOUR TARGET HOST HERE OR VIA COMMAND LINE !!!
    host = "http://localhost:8080"

    # Wait time between tasks (adjusted for high throughput)
    # A low wait_time maximizes the Requests Per Second (RPS) per user.
    # Set this to a low value or zero (constant(0)) if you need max RPS.
    wait_time = between(0.01, 0.1)

    @task
    def submit_random_xml_request(self):
        """Executes a POST request with a randomly chosen XML payload."""

        if not PAYLOADS:
            # This should ideally not happen if files are loaded correctly
            return

        selected_payload = random.choice(PAYLOADS)
        endpoint_path = "/api/xml-processor"

        # The 'name' parameter sets the name shown in the Locust reports
        response = self.client.post(
            endpoint_path,
            data=selected_payload,
            headers=HEADERS,
            name="Submit_Order_XML_Request"
        )

        # --- Optional: Response Validation ---
        if response.status_code != 200:
            response.failure(f"Request failed with status code: {response.status_code}")

-----

## Running for 900k RPM Ramp-up

A ramp-up to **900,000 RPM (Requests Per Minute)** in 1 minute means you need an average of **15,000 Requests Per Second (RPS)** at the end of the first minute.

This level of load is **extremely high** and typically **requires a distributed setup** using multiple load generator machines (Locust workers) to avoid hitting CPU/network limits on a single machine.

### 1\. Calculate Required Users

Assuming your application and network can achieve an average of **15 RPS per concurrent user** (a very high estimate for complex XML/SOAP requests), you would need:

$$\text{Total Users} = \frac{15,000 \text{ RPS}}{15 \text{ RPS per User}} = 1,000 \text{ concurrent users}$$

However, since a single user often generates much less than 15 RPS, you will likely need **several thousand concurrent users** (e.g., 5,000 to 10,000) to reach 15,000 RPS.

### 2\. Calculate Hatch Rate

To reach a target of, say, **6,000 users** in **60 seconds (1 minute)**, your required hatch rate is:

$$\text{Hatch Rate} = \frac{\text{Total Users}}{\text{Ramp-up Time (seconds)}} = \frac{6,000}{60} = 100 \text{ users/second}$$

### 3\. Command for Headless Distributed Run

To run this scenario, you must use Locust in **distributed mode** (Master/Worker setup) and run it **headlessly** (without the web UI) with the calculated parameters:

#### A. Start the Locust Master Node

Run this command on your Master machine:

```bash
locust -f locustfile.py --master --host http://<YOUR_TARGET_HOST>
```

#### B. Start Multiple Locust Worker Nodes

Run this command on **multiple** load generator machines (as many as needed to handle the load):

```bash
locust -f locustfile.py --worker --master-host <MASTER_IP_ADDRESS>
```

#### C. Start the Test on the Master

Once all workers are connected, initiate the test from the Master machine using the following parameters:

| Parameter | Value | Description |
| :--- | :--- | :--- |
| `--headless` | | Runs without the web interface. |
| `--users` | **6000** | The total number of users to spawn (adjust based on required RPS). |
| `--hatch-rate` | **100** | The number of users to spawn per second. |
| `--run-time` | **1m10s** | Run the test for 1 minute and 10 seconds (to capture the full 1-minute ramp-up). |
| `--host` | `http://your.service.com` | **CRITICAL:** The base URL of the system under test. |

```bash
locust -f locustfile.py --master --headless --users 6000 --hatch-rate 100 --run-time 1m10s --host http://your.service.com
```

-----

**⚠️ Performance Warning:** Reaching 900,000 RPM is an extreme load. You will need to monitor the CPU, memory, and network usage of your load generators closely. If the Locust worker machines run out of resources, they become the bottleneck, and the test results will not accurately reflect your service's performance.